
This notebook sets up and runs a series of analysis stages that are dispatched over the compute nodes of a multiprocessing environment, either locally (using the `multiprocessing` Python module) or remotely onto a *slurm*/*singularity* enabled computer cluster.

The first 5 notebook cells show how to set up a common pipeline that is actually launched at the 6th cell of code, calling the `run` method of an `RWAnalyzer` object.
As explained below, the first part of the notebook is dispatched and run elsewhere. The call to `run` marks where this active part stops.
As a consequence, the notebook is structured as follows:

* the first section features code cells that implement the pipeline, plus a few optional cells that should not interfere with the pipeline,
* the second section - code cells after `a.run()` - dissects some specific parts of the pipeline and manipulate the generated output,
* the last section gives a cleaned-up version of the sensitive part of the pipeline, to make clear which lines of code are essential.

# A simple *tessellate and infer* pipeline to resolve the diffusivity and effective potential in space

The main difficulty in dispatching computations on remote hosts lies in locating the data files.
An approach that has been favored in the use cases developped as of version `0.5` consists of making all the paths absolute.

In [2]:
import os

wd = '~/' + os.path.relpath(os.getcwd(), os.path.expanduser('~')).replace('\\', '/')
wd

'~/github/TRamWAy/notebooks'

We set up an `RWAnalyzer` object with SPT data files, the corresponding files for regions of interest,
the segmentation to be applied to each ROI, and the inference procedure to be applied to each microdomain.

In [5]:
from tramway.analyzer import *

a                                 = RWAnalyzer()

a.spt_data.from_ascii_files(f'{wd}/data-examples/*.rpt.txt')
a.spt_data.localization_precision = 0.03

a.roi.from_ascii_files(suffix='roi') # => *.rpt-roi.txt

a.tesseller                       = tessellers.Hexagons

a.mapper.from_plugin('stochastic.dv')
a.mapper.diffusivity_prior        = 20
a.mapper.potential_prior          = 1
a.mapper.max_runtime              = 100 # in seconds; 100 seconds is much too short for a proper DV estimation, but convenient for a quick example
a.mapper.verbose                  = False
a.mapper.worker_count             = 4

Let us make sure the paths are alright.
Of note, this notebook assumes the [introduction notebook](RWAnalyzer%20tour.ipynb) ran before, or at least its very first code cell, so that the input data are available.

In [4]:
a.spt_data.filepaths

['/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt',
 '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.txt']

In [5]:
[ f.roi.filepath for f in a.spt_data ]

['/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt-roi.txt',
 '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt-roi.txt']

Below are defined the different pipeline stages, using building blocks available in the `stages` module exported by the `tramway.analyzer` package.

In [6]:
a.pipeline.append_stage(stages.tessellate())
a.pipeline.append_stage(stages.reload())
a.pipeline.append_stage(stages.infer())

The declarative approach to setting a pipeline grants control over how to run the pipeline.

The aim of such a design consisted in making such inferences run on computer clusters, in particular Slurm- and singularity-enabled clusters, communicating with these clusters through an SSH connection.

At *Institut Pasteur* in Paris, France, the main computer cluster was baptised *Maestro*.
The `environments` module exported by the `tramway.analyzer` package features a `Maestro` predefined environment that can be used to set the `env` attribute of an `RWAnalyzer` object.

This environment object takes the local username to connect to the *Maestro* submit node. This can be overriden with the `username` attribute, for example in `a.env.ssh.username`.

However, the Maestro cluster is accessible only over *Institut Pasteur*'s VPN or from the campus.

To make this notebook run in more different circumstances, we will use the `LocalHost` environment instead.
This environment operates in a similar fashion, but on the local computer, and does not involve any remote resource.

It actually offers a convenient way to test a pipeline before running the same pipeline on a computer cluster.

In [7]:
#a.env                             = environments.Maestro # works only over Institut Pasteur's VPN or on campus
a.env                             = environments.LocalHost # replacement so that the demo can work anywhere
a.env.worker_count                = 10

a.env.script                      = 'RWAnalyzer standard pipeline.ipynb'

If someday you may export this notebook as a regular Python script, you should consider using the `__file__` variable, as demonstrated in the following code cell, as a replacement for the last line above. Otherwise, you can safely delete the following cell:

In [8]:
try:
    a.env.script                  = __file__
except NameError:
    # in an IPython notebook, `__file__` is not defined and there is no standard way to get the notebook's name
    a.env.script                  = 'RWAnalyzer standard pipeline.ipynb'

The following cell is optional. The default logging level is `INFO`.

In [9]:
import logging

a.logger.setLevel(logging.DEBUG)

The `run` method launches the pipeline.
The workload is concentrated in the following code cell:

In [10]:
a.run()

working directory: /tmp/tmpy4puqf8h
setup complete
running: jupyter nbconvert --to python "/home/flaurent/github/TRamWAy/notebooks/RWAnalyzer standard pipeline.ipynb" --stdout
initial dispatch done
jobs ready
submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=0 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt"
submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=0 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.txt"
jobs submitted
setup complete
stage 0 ready
tessellating roi: 'roi000' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt')...
tessellating roi: 'roi001' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt')...
tessellating roi: 'roi002' (in source '/h

submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=2
submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=3
submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=4
submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=5
submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8l

setup complete
stage 2 ready
inferring on roi: 'roi015' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa')...
stage 2 done
job 15 done

submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=25
setup complete
stage 2 ready
inferring on roi: 'roi016' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa')...
stage 2 done
job 16 done

submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa" --region-index=26
setup complete
stage 2 ready
inferring on roi: 'roi017' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-0

job 34 done

submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa" --region-index=8
setup complete
stage 2 ready
inferring on roi: 'roi035' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa')...
stage 2 done
job 35 done

submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa" --region-index=9
setup complete
stage 2 ready
inferring on roi: 'roi000' (in source '/home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa')...
stage 2 done
job 36 done

submitting: /usr/bin/python3 /tmp/tmpy4puqf8h/tmpw8lydy_7.py --working-directory="/tmp/tmpy4puqf8h" --stage-index=1,2 --source="/home

reading file: /tmp/tmpy4puqf8h/tmp03big2i3.rwa
for source file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.txt...
writing file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-02-15ms.rpt.rwa
for source file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.txt...
writing file: /home/flaurent/github/TRamWAy/notebooks/data-examples/Manip01-01-Beta400AA-01-15ms.rpt.rwa
results collected


At this point, the pipeline is complete. As many *.rwa* files as input SPT data files were generated both on the local and remote hosts (if different).

### Sequence diagram

The procedure for distributing computations using the `LocalHost` environment is similar to (slightly simpler than) the `Maestro` environment which is a specialized `SlurmOverSSH` environment:

![Sequence diagram](SlurmOverSSH.svg)

The above sequence diagram is available [here](https://sequencediagram.org/index.html#initialData=CoSwLgNgpgBACiADlCIB2soA8oGMCuYIA9mjAO7gAWMAyhPgE4C2A8gG5SO20ASMUNOxCNSzQWADOAKGmIAhoyK4k8tGADmo-IhgBiVBqqbGUQTABUF2vgBGzcDEkgAJlCtzFy1epgAiACkdAE8wLgAdNAAySLRiMNtiYgBrPxh5SRgAzyUQFQVfPxt7R0RRXChJSTSMmAAZaUEXHO8Ck2IdfUNjWwZYKwB1YkZkridXdwsWvJ8wfwBZDLDGGDi3Gsz56fy1OaKGFg2YACVt2f8hkbG1qCOBxrRm6QCAWgA+CzqALhPKsC8YFETvgyABBCAQaR1d4BH5oWy4UicJRQgA8LxesJgAFVJGNJLhGEgwFD3vMfgBhKh4ZIwRHqeToCJoeTsRkQeS2ECoMDBaTzMk-AAixHIaAgxHkLhgIAAZjAHFV0BppPJcEQ2WEYFs3GqNfIwvJZcttaS3uSYBTTAbYORhsllTAXCI8GBhnzoebhSBJAowLgaPg8SsABSIYIASichOJ0i0HV0suGAjVNERzEQxGcWsk-w0UDN32BZES8VzjHkulz8nzMvlaj5epAmtgDV16ubNqNJoanotQp9foDTlsBuHIckVCjBKJiBJnuOP2OIJHY5oM9jx3eAyXK6D+Jjc8i6DppH+TMYkUoYBo-0ktMUGnw4nUMgG25+HC4RLcMA6YEQQhIkYch5BgWVuSgP0qDfD9ixgUspDACtEEiatazldI0EbDsWxge5231Q1jTGe53zeHd4LA5w0A0aAYDvZJVVwm18LjbRdAlYhEDNRdLWpXBaWrMAgzRDFF2edEXiLY4oOGed3iLAZGRJJp+Skyi4C8EB5AgGAQLAiD6LUaV90YF43AgjBpURCFXRAXpKmkQjOy1e4+29X01yw6VGBBSJbOgDtSBgMNI2jWcSQFL1LWIZguQwGAFFyXT9NA8DIMyEyAutLVkEYZxcwkNLDMy5iiNgLY6iki06h9OZiHlfLCrCXxAvsxyZBcvCqsFX5kJAKBOHSCFTzsjtOvK1zWzEl5yWcqAmzwho1ICKTviAA).

### Input data files are not dispatched

The above diagram calls for a long explanation but, first of all, the most important point here is that scripts and executables only are dispatched, and the **input data are NOT dispatched** onto the worker side.

This means that the user has to prepare the data on both the submit and worker sides.
This includes SPT data files and ROI files.

These input data files must be located (or reachable) at the same paths on both sides, which is not trivial.
The recommended approach consists of locating the data either from the filesystem root (/) or home/user directory.
To make absolute paths easier to translate from the local filesystem to the remote filesystem, special care was taken with the '~' shortcut that refers to the home/user directory.

### The *reload* bootstrap stage

While the `tessellate` and `infer` stages have explicit goals, the `reload` stage might look optional.
Indeed, this stage is optional if no so-called *environments* are defined.
In this case, the stages are sequentialy run in the notebook kernel.

In all the parallelizing settings, the analysis tree is updated by the `tessellate` stage only in the processes that actually ran this stage.
On the local (or submit) side that delegates this stage, the local RWAnalyzer object is not aware of the availability of such an update.
However, this update is required for the next stage.
Not only is this update required for the next stage to be fed with some input, it is also required on the submit side to determine the number of and command-line arguments for the tasks that will run the `infer` stage.

As a consequence, because the `tessellate` stage has updated the analysis trees in the already-existing or newly-created *.rwa* files, the `spt_data` attribute must be redefined so that it will load the updated *.rwa* files before the `infer` stage is scheduled.

Unlike the `tessellate` and `infer` stages, the `reload` stage must be performed both on the submit and worker sides and is classified as a *bootstrap* stage.
Every worker runs this bootstrap stage before running the assigned `infer` task.

Note that the `reload` stage is also expected to generate significantly less workload than the other two stages.

### Stage granularity

\[To be continued\]